In [1]:
import numpy as np
import pandas as pd

In [2]:
node = pd.read_table("./node.txt",delimiter=",")
node["osmid"].astype("int64")
node = node[["osmid","y","x"]]
node

,osmid,y,x
0,2.558506e+07,39.901987,116.385578
1,2.558508e+07,39.899003,116.386792
2,2.558509e+07,39.898860,116.381862
3,2.558512e+07,39.900544,116.377850
4,2.558512e+07,39.902940,116.377763
...,...,...,...
37481,1.007778e+10,39.996019,116.222735
37482,1.009133e+10,39.896058,116.599433
37483,1.009133e+10,39.773362,116.173259
37484,1.009133e+10,39.775494,116.169075


In [3]:
node_info = {}
num = 0
for i in range(node.shape[0]):
    # print(round(node["osmid"][i]))
    node_info[round(node["osmid"][i])] = [num,node["y"][i],node["x"][i]]
    num = num+1

In [22]:
print(node_info[8597854497])  #  11065
print(node_info[1838508278]) # 11066

[29164, 39.9464039, 116.4604351]
[14684, 39.9469905, 116.4625124]


In [5]:
edge = pd.read_table("./edge.txt",delimiter=",")
edge = edge[["FID","u","v"]]
# node["osmid"].astype("int64")
edge

/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,FID,u,v
0,0,2.558506e+07,5.336507e+08
1,1,2.558506e+07,8.835881e+09
2,2,2.558506e+07,3.427913e+08
3,3,2.558508e+07,1.414035e+09
4,4,2.558508e+07,9.378036e+09
...,...,...,...
86311,86311,1.009133e+10,4.076207e+09
86312,86312,1.009133e+10,5.345749e+08
86313,86313,1.009133e+10,9.783252e+09
86314,86314,1.009133e+10,1.009133e+10


In [6]:
# np.unique(edge["u"].to_numpy()).shape
print(edge["FID"][32428])
print(edge["u"][32429])
print(edge["u"][32427])


32428
1819087328.0
1819087316.0


### 处理经过路网匹配后的数据 得到长度大于等于15 小于15 等于1的轨迹

In [7]:
import os
path = "./bj-output-omp-clean/"
files = os.listdir(path)
# files.sort(key=lambda x:int(x.split('.')[0]))

record_total = 0
len_total = 0

max_len = 0
min_len = 10000


# 保存子轨迹 大于等于15
traj_up15 = []
traj_low15 = []
traj_1 = []

# exit()
for file in files:
    # print(file)
    sub_traj_mapped = pd.read_csv(path+"{}".format(file),delimiter=";")
    record_total = record_total + sub_traj_mapped.shape[0]

    for index,row in sub_traj_mapped.iterrows():
        if len(row[0].split("\""))==5:
            if len(row[0].split("\"")[1].split(","))>=15:
                traj_up15.append(row[0].split("\"")[1]+","+file.split(".")[0])  # 加上车辆ID
            else:
                traj_low15.append(row[0].split("\"")[1]+","+file.split(".")[0])  # 加上车辆ID
        else:
            traj_1.append(row[0].split("\"")[0].split(",")[1]+","+file.split(".")[0])

In [31]:
print(len(traj_up15))  # ['82397,352,32434,297,35115,33296,33299,1736,609,19166,86233,611,85744,26248,26247,33309,26245,26243,10'
print(len(traj_low15))
print(len(traj_1))
record_total

19111
17583
1513


38207

In [33]:
np.save("./traj/traj_up15.npy",np.array(traj_up15))
np.save("./traj/traj_low15.npy",np.array(traj_low15))
np.save("./traj/traj_1.npy",np.array(traj_1))

### 为路段加入其它特征（路段的起始和终点） 方便后续处理

In [ ]:
edgeid_node_dir_map = np.load("./edge_info/edgeid_node_dir_map_eid.npy",allow_pickle=True).tolist()
len(edgeid_node_dir_map)
traj_low15 = np.load("./traj/traj_low15.npy",allow_pickle=True)
traj_1 = np.load("./traj/traj_1.npy",allow_pickle=True)

id_ndoe_dir_traj_low15 = []
id_ndoe_dir_traj_1 = []

for item in traj_low15:
    sub_traj = [int(i) for i in item.split(",")]

    temp = []
    for edgeid in sub_traj[:-1]:
        # print(edgeid)
        # print(edgeid_node_dir_map[edgeid])
        temp.append(np.append(edgeid_node_dir_map[edgeid],sub_traj[-1]))
    id_ndoe_dir_traj_low15.append(temp)

for item in traj_1:
    sub_traj = [-1 if len(i.split("."))==2 or item.split(",")[0]==""  else int(i)  for i in item.split(",")]
    temp = []
    for edgeid in sub_traj:
        if(edgeid != -1):
            temp.append(np.append(edgeid_node_dir_map[edgeid],sub_traj[-1]))
    id_ndoe_dir_traj_1.append(temp)

In [26]:
np.save("./traj/id_ndoe_dir_traj_low15.npy",np.array(id_ndoe_dir_traj_low15))
np.save("./traj/id_ndoe_dir_traj_1.npy",np.array(id_ndoe_dir_traj_1))

print(len(id_ndoe_dir_traj_low15))
print(len(id_ndoe_dir_traj_1))

/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


17583
1513


/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


### 从up15的轨迹中拆分出训练集和测试集,并为每个路段加入其它特征

In [10]:
edgeid_node_dir_map = np.load("./edge_info/edgeid_node_dir_map_eid.npy",allow_pickle=True).tolist()
traj_up15 = np.load("./traj/traj_up15.npy",allow_pickle=True)
print(traj_up15[:5])
rng = np.random.default_rng(12345)
# 打乱数据顺序
rng.shuffle(traj_up15)
print(traj_up15[:5])
# 划分训练集和测试集
trainsize = round(len(traj_up15)*0.9)
traj_up15_train = traj_up15[:trainsize]
traj_up15_test = traj_up15[trainsize:]



id_ndoe_dir_traj_train = []
id_ndoe_dir_traj_test = []
for item in traj_up15_train:
    
    sub_traj = [int(i) for i in item.split(",")]

    temp = []
    for edgeid in sub_traj[:-1]:
        # print(edgeid)
        # print(edgeid_node_dir_map[edgeid])
        temp.append(np.append(edgeid_node_dir_map[edgeid],sub_traj[-1]))
    id_ndoe_dir_traj_train.append(temp)
    # break

for item in traj_up15_test:
    
    sub_traj = [int(i) for i in item.split(",")]

    temp = []
    for edgeid in sub_traj[:-1]:
        # print(edgeid)
        temp.append(np.append(edgeid_node_dir_map[edgeid],sub_traj[-1]))
    id_ndoe_dir_traj_test.append(temp)

['39330,39326,32243,20272,35997,39576,20273,41961,41956,41953,41951,18688,17174,85327,85332,82540,5494,32570,1851,48609,48604,5327'
 '1391,76096,37609,5300,648,14723,14774,5872,5868,5876,5848,5242,5238,5897,27174,5894,14586,14494,5884,9118,4545,4563,3536,14441,83179,83181,6519,8566,5368,34905,27187,8567,8513,21127,21120,21129,21122,21109,21134,8507,4555,208,75889,25802,25816,4464,32830,36170,18020,5474'
 '548,31030,702,30985,30963,30959,48913,30956,31024,31021,31014,31003,30997,31002,32411,31004,31037,9344,31047,31048,31050,8517'
 '31142,31139,31137,14236,31138,51216,29742,634,6594,24319,108,112,126,124,123,8517'
 '2713,1188,29467,2172,35766,2177,2171,2175,12069,26022,2182,24801,2226,21146,21137,26886,26885,4413']
['67134,33087,55579,64482,64480,64474,64481,64478,64476,34569,64805,30238,34564,34567,34570,3605,34572,919'
 '24386,511,27401,532,537,21532,21514,53771,53769,27537,27535,44340,3881,27533,33789,3882,5436,39732,5219,6315,5221,26126,17351,26116,33779,33778,26115,26117,26124,2613

In [11]:
np.save("./traj/id_ndoe_dir_traj_from_up15_train.npy",np.array(id_ndoe_dir_traj_train))
np.save("./traj/id_ndoe_dir_traj_from_up15_test.npy",np.array(id_ndoe_dir_traj_test))

print(len(id_ndoe_dir_traj_test))
print(len(id_ndoe_dir_traj_train))

/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


1911
17200


In [ ]:
id_ndoe_dir_traj_test[-10]

## 分割 滑动窗口为5

In [13]:
traj_dataset_train = []
z = 5
for item in id_ndoe_dir_traj_train:
    # print(item)
    # print(len(item))
    """
    edgeid,node_v,node_id,angle,象限
    [[39330, 2382886309, 17409, 270, 6],....]
            21
    """
    # break
    for i  in range(len(item)):
        # print(item[i*z:15+i*z])
        if(len(item[i*z:15+i*z])==15):
            traj_dataset_train.append(item[i*z:15+i*z])
    # break

In [14]:
np.save("./dataset/traj_dataset_train.npy",np.array(traj_dataset_train))
np.array(traj_dataset_train).shape

(91687, 15, 6)

In [15]:
traj_dataset_test = []
z = 5
for item in id_ndoe_dir_traj_test:
    # print(item)
    # print(len(item))
    """
    edgeid,node_v,node_id,angle,象限
    [[39330, 2382886309, 17409, 270, 6],....]
            21
    """
    # break
    for i  in range(len(item)):
        # print(item[i*z:15+i*z])
        if(len(item[i*z:15+i*z])==15):
            traj_dataset_test.append(item[i*z:15+i*z])

In [16]:
np.save("./dataset/traj_dataset_test.npy",np.array(traj_dataset_test))
np.array(traj_dataset_test).shape

(10348, 15, 6)

In [ ]:
traj_dataset_test[-10]

In [17]:
# (102035, 15, 6)  edgeid,node_v,node_id,angle,象限,taxiid

## 路网约束 node_num * 8 

In [26]:
edge_dir_emb = pd.read_csv("./1114/edge_dir_emb-t.csv").to_numpy()
print(edge_dir_emb)
"""
[[0, 25585055, 0, 533650673, 4046, 106, 2],
 [1, 25585055, 0, 8835880543, 29680, 359, 7],
 [2, 25585055, 0, 342791346, 2488, 191, 4],
 [3, 25585083, 1, 1414035094, 11091, 317, 7],
 [4, 25585083, 1, 9378036221, 33037, 268, 5],
 [5, 25585092, 2, 1414035092, 11090, 1, 0],
 [6, 25585092, 2, 2099138477, 16079, 267, 5],
"""
node_mask = {}
for item in edge_dir_emb:
    # print(item)
    if item[1] not in node_mask:
        letter = [0 for _ in range(8)]
        letter[item[4]] = 1
        node_mask[item[1]] = letter
    else:
        node_mask[item[1]][item[4]] = 1
    
    # break

[[    0     0  4046   106     2]
 [    1     0 29680   359     7]
 [    2     0  2488   191     4]
 ...
 [86313 37484 36319    36     0]
 [86314 37484 37485   127     2]
 [86315 37485 37484   307     6]]


In [27]:
node_mask
np.save("./1114/node_mask.npy",node_mask)

In [ ]:
dataset = np.load("./1114/traj_dataset_window5_5fea.npy",allow_pickle=True)

In [ ]:
node_mask = np.load("./1114/node_mask.npy",allow_pickle=True)
node_mask.tolist().keys()